In [97]:
import numpy as np
import copy
import sys
from collections import defaultdict

class game():
    def __init__(self, N_, isBadSide_):
        
        assert N_ == len(isBadSide_), "invalid initialization"
        
        self.N = N_
        self.isBadSide = isBadSide_
        self.p_loss = sum(self.isBadSide)/self.N
        #self.s = 0
        self.actions = {'play','stop'}
        
    def R(self, s, s_next):
        """reward depend on s and s_next"""
        if s=='end' or s=='loss':
            return 0
        if s_next == 'end':
            return 0
        elif s_next == 'loss':
            return -s
        else:
            return s_next - s
        
    def T(self, s, a):
        """return a dict of {s_next: prob}"""
        if s == 'end' or s=='loss' or a=='stop':
            return {'end': 1}
        
        return {s_next: self.p_loss if s_next=='loss' else 1.0/self.N for s_next in self.possible_next_states(s,a)}
        
    def possible_next_states(self, s, a):
        """return possible next states given s and a"""
        if a == 'stop':
            return ['end']
        else:
            return ['loss']+[s+i+1 for i in range(self.N) if self.isBadSide[i]==0]
        
        
        
#     def move(self, action):
#         R = self.R[self.s]
        
#         next_s = list(self.T[self.s, action].keys())
#         p = list(self.T[self.s, action].values())
#         #print(next_s, p)
#         i = np.random.choice(len(next_s), p=p)
#         self.s = next_s[i]
        
#         return R
    
#     def reset_pos(self):
#         self.s = list(self.states)[np.random.randint(len(self.states))]

In [98]:
g = game(21, [1,1,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,1,0])
g = game(22, [1,1,1,1,1,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,1,0])
g = game(6, [1, 1, 1, 0, 0, 0])

In [99]:
V, p = value_iteration(g, max_iteration=1000, beta=1)
aws = V[0]
print(aws)

value iteration converged at 3
2.583333333333333


In [100]:
# The state is the current balance
# We don't have a fix number of states in this problem, so we will start from s=0, and adding possible states to Value func dict

def value_iteration(game, max_iteration=1000, beta=1):
    # initialize
    V = defaultdict(int)
    V[0] = 0
    
    for n in range(max_iteration):
        V_new = defaultdict(int)
        explored_states = list(V.keys())
        for s in explored_states:
            V_new[s] = max(sum((game.R(s,s_next)+beta*V[s_next])*T for s_next, T in game.T(s,a).items()) for a in game.actions)

        if all(abs(V_new[s]-V[s])<1e-8 for s in explored_states):
            print(f"value iteration converged at {n}")
            break
        V.update(V_new)
        
    p = {}
    
    for s in explored_states:
        q = {sum((game.R(s,s_next) + beta*V[s_next])*T for s_next, T in game.T(s,a).items()): a for a in game.actions}
        #print(s, q)
        p[s] = q[max(q.keys())]
    return V, p